In [6]:
import os
import pandas as pd
import pymongo


import requests
from requests_oauthlib import OAuth1
import json

from __future__ import print_function

In [7]:
DB_NAME = 'amz_local_search'
MONGO_URI = os.environ.get('AMZ_MONGO_URI_TEMPLATE') % (os.environ.get('AMZ_MONGO_PASSWORD'), DB_NAME)

client = pymongo.MongoClient(MONGO_URI)
db = client.get_database()

In [8]:
OUT_DIR = '.activities/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

# Load WooCommerce customers

In [9]:
WC_DAGA_DIR = '.woo_data/'

with open(os.path.join(WC_DAGA_DIR, 'wc_customers.json'), 'r') as f:
    raw_wc_customers = json.load(f)
len(raw_wc_customers)

814

In [10]:
def enum_wc_cusotmers():
    for e in raw_wc_customers:
        yield {
            'wc_user_id': e.get('id'),
            'wc_email': e.get('email'),
            'wc_user_created': e.get('date_created'),
            'wc_is_paying': e.get('is_paying_customer'),
            'wc_role': e.get('role')            
        }
wc_customers_df = pd.DataFrame(enum_wc_cusotmers())
wc_customers_df.head(1)

,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id
0,138213802@qq.com,False,customer,2020-04-13T20:34:01,17874


In [11]:
print ('wc_customers_df.is_paying_customer=True: ', len(wc_customers_df[wc_customers_df['wc_is_paying']]))

wc_customers_df.is_paying_customer=True:  221


In [12]:
wc_customers_df[wc_customers_df['wc_user_id'] == 17530]

,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id


# Load WooCommerce orders

In [13]:
with open(os.path.join(WC_DAGA_DIR, 'wc_orders.json'), 'r') as f:
    raw_wc_orders = json.load(f)
len(raw_wc_orders)

1037

In [14]:
def enum_wc_orders():
    for order in raw_wc_orders:
        for line_item in order.get('line_items', []):
            yield {
                'wc_user_id': order.get('customer_id'),
                'wc_ip_address': order.get('customer_ip_address'),
                'wc_order_total': float(order.get('total')),
                'wc_item_total': float(line_item.get('total')),
                'wc_product': line_item.get('name')
            }
wc_order_items_df = pd.DataFrame(enum_wc_orders())
wc_order_items_df.head(2)

,wc_ip_address,wc_item_total,wc_order_total,wc_product,wc_user_id
0,74.213.72.12,25.0,25.0,Subscription - Basic,18026
1,108.7.207.84,200.0,210.0,Subscription - Premium+,18025


In [15]:
wc_orders_df = wc_order_items_df.groupby('wc_user_id').agg({
        'wc_ip_address': { 'wc_ip_address': 'last' },
        'wc_product': { 'wc_products': set }, 
        'wc_item_total': { 'wc_item_total': 'sum', 'wc_payments': list }
    })

wc_orders_df.columns = wc_orders_df.columns.droplevel(0)
wc_orders_df['wc_products'] = wc_orders_df['wc_products'].map(', '.join)
wc_orders_df['wc_payments'] = wc_orders_df['wc_payments'].map(str)
wc_orders_df.sort_values(['wc_item_total'], ascending=[False], inplace=True)
wc_orders_df.head(10)

/Users/evgenyp/Projects/AmzJet-ZT/zt-amzjet-backend/amz-deploy/dev/.venv-dev/lib/python2.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,wc_products,wc_item_total,wc_payments,wc_ip_address
wc_user_id,,,,
17436,Audit - 1-3M,3000.00,[3000.0],107.77.231.4
17413,"Feature: Volatility, variability, Subscription...",2093.00,"[200.0, 49.0, 200.0, 49.0, 200.0, 49.0, 200.0,...",136.49.90.82
17827,"Feature: ASINs included in keyword - 10 ASINs,...",967.00,"[858.0, 10.0, 99.0]",12.186.143.218
17650,"Feature: Rank decimal places (new), Subscripti...",957.00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",115.189.88.197
17500,"Subscription - Basic, Feature: ASINs included ...",827.25,"[21.25, 8.5, 21.25, 8.5, 21.25, 8.5, 21.25, 8....",74.88.44.149
17849,"Subscription - Premium, Feature: Rank decimal ...",779.00,"[100.0, 0.0, 100.0, 0.0, 100.0, 0.0, 100.0, 0....",172.250.11.175
17702,"Feature: ASINs included in keyword - 10 ASINs,...",649.00,"[50.0, 5.0, 0.0, 50.0, 5.0, 0.0, 50.0, 5.0, 0....",41.157.114.57
18025,"Feature: ASINs included in keyword - 10 ASINs,...",630.00,"[200.0, 10.0, 200.0, 10.0, 200.0, 10.0]",108.7.207.84
17914,Subscription - Premium+,600.00,"[200.0, 200.0, 200.0]",136.49.46.161


# Fetch data from Actions collection into a DataFrame

In [16]:
raw_actions_df = pd.DataFrame(db.actions.find({}))
raw_actions_df.head(1)

,_id,action_sig,args,occurred_at,stat,state,tracking,updated_at,user
0,1-zAStaPChTkVwWNK7j7qNKg,"amz_local_search.report(query, asin)","{u'query': u'carabiner clip', u'start_date': 2...",2020-10-20 20:59:50.336,NaN,{u'msg': u'ASIN lookup history is empty for th...,"{u'ip': u'127.0.0.1', u'ua': u'Mozilla/5.0 (Ma...",NaT,{u'id': 1}


In [19]:
actions_df = raw_actions_df[['action_sig', 'occurred_at', 'updated_at']].copy(deep=False)
actions_df['action_id'] = raw_actions_df['_id']
actions_df['user_id'] = raw_actions_df['user'].map(lambda u: u.get('id'))
actions_df['succeeded'] = raw_actions_df['state'].map(lambda s: s.get('success'))
actions_df['done'] = raw_actions_df['state'].map(lambda s: s.get('done'))
actions_df['arg_query'] = raw_actions_df['args'].map(lambda a: a.get('query'))
actions_df['arg_asin'] = raw_actions_df['args'].map(lambda a: a.get('asin'))
actions_df['error_type'] = raw_actions_df['state'].map(lambda a: a.get('error', {}).get('type_id'))
actions_df['error_msg'] = raw_actions_df['state'].map(lambda a: a.get('error', {}).get('msg'))

actions_df['occurred_at_day'] = actions_df['occurred_at'].dt.to_period('D')
actions_df['occurred_at_week'] = actions_df['occurred_at'].dt.to_period('W')
actions_df['occurred_at_month'] = actions_df['occurred_at'].dt.to_period('M')

actions_df.head(5)

,action_sig,occurred_at,updated_at,action_id,user_id,succeeded,done,arg_query,arg_asin,error_type,error_msg,occurred_at_day,occurred_at_week,occurred_at_month
0,"amz_local_search.report(query, asin)",2020-10-20 20:59:50.336,NaT,1-zAStaPChTkVwWNK7j7qNKg,1,None,None,carabiner clip,@latest,None,None,2020-10-20,2020-10-19/2020-10-25,2020-10
1,amz_local_search.search,2020-10-20 21:02:29.573,2020-10-20 21:03:23.131,1-BnQ7RRN8SJQscjRUEJXDx7,1,True,True,coloring book,None,None,None,2020-10-20,2020-10-19/2020-10-25,2020-10
2,amz_local_search.search,2020-10-20 21:04:22.726,2020-10-20 21:05:17.127,1-E8Yy4PpDrVvxtaVNt9MrhM,1,True,True,another keyword,None,None,None,2020-10-20,2020-10-19/2020-10-25,2020-10
3,"amz_local_search.report(query, asin)",2020-10-20 21:04:50.679,NaT,1-PfStroFkDubVVHcpgRxTrk,1,None,None,carabiner clip,@latest,None,None,2020-10-20,2020-10-19/2020-10-25,2020-10
4,"amz_local_search.report(query, asin)",2020-10-20 21:05:10.952,NaT,1-tRdCSkhyAt6Rkvjvi36CAR,1,None,None,coloring book,@latest,None,None,2020-10-20,2020-10-19/2020-10-25,2020-10


# Join actions with other metadata

In [20]:
full_actions_df = actions_df.merge(wc_customers_df, how='left', left_on='user_id', right_on='wc_user_id')
full_actions_df = full_actions_df.merge(wc_orders_df, how='left', left_on='user_id', right_on='wc_user_id')
full_actions_df.head(1)

,action_sig,occurred_at,updated_at,action_id,user_id,succeeded,done,arg_query,arg_asin,error_type,...,occurred_at_month,wc_email,wc_is_paying,wc_role,wc_user_created,wc_user_id,wc_products,wc_item_total,wc_payments,wc_ip_address
0,"amz_local_search.report(query, asin)",2020-10-20 20:59:50.336,NaT,1-zAStaPChTkVwWNK7j7qNKg,1,None,None,carabiner clip,@latest,None,...,2020-10,NaN,NaN,NaN,NaN,NaN,"Subscription - Basic, Feature: ASINs included ...",141.5,"[25.0, 0.0, 12.5, 5.0, 99.0, 0.0]",24.19.170.196


In [21]:
print('Unmapped user id-s:')
full_actions_df[pd.isnull(full_actions_df.wc_email)].user_id.unique()

Unmapped user id-s:


array([    1, 17245, 17277, 17278, 17301, 17345, 17530, 17856, 18031,
       18045,     2])

# Insights

In [22]:
actions_per_user_df = full_actions_df.groupby('user_id').agg({
        'wc_email': { 'wc_email': 'last' },
        'wc_is_paying': { 'wc_is_paying': 'last' },
        'arg_query': { 'num_keywords': 'nunique' }, 
        'arg_asin': { 'num_asins': 'nunique' }, 
        'occurred_at_month': { 'num_months_active': 'nunique' },    
        'occurred_at_week': { 'num_weeks_active': 'nunique' },
        'occurred_at_day': { 'num_days_active': 'nunique', 'last_day_seen': 'last' },
        'action_id': { 'num_requests': 'count' },
        'wc_user_created': { 'registered_at': 'last' },
        'wc_products': { 'orders': 'last' },
        'wc_item_total': { 'orders_total': 'last' },
        'wc_payments': { 'payments': 'last' },
        'wc_ip_address': { 'ip_address':  'last' }
    })

actions_per_user_df.columns = actions_per_user_df.columns.droplevel(0)
actions_per_user_df.sort_values(['num_weeks_active', 'num_days_active'], ascending=[False, False], inplace=True)
actions_per_user_df.head(100)

,num_weeks_active,num_months_active,registered_at,last_day_seen,num_days_active,orders,wc_is_paying,ip_address,payments,num_asins,orders_total,wc_email,num_keywords,num_requests
user_id,,,,,,,,,,,,,,
17457,50,12,2019-11-05T08:20:46,2020-10-25,235,"Feature: ASINs included in keyword - 10 ASINs,...",True,79.183.98.7,"[12.75, 8.5, 12.75, 8.5, 12.75, 8.5, 12.75, 8....",64,255.00,Hen@Expander.co,297,20444
17413,41,11,2019-10-28T02:28:58,2020-05-03,261,"Feature: Volatility, variability, Subscription...",True,136.49.90.82,"[200.0, 49.0, 200.0, 49.0, 200.0, 49.0, 200.0,...",38,2093.00,tkillian@connectivonline.com,227,320507
17376,41,13,2019-10-25T14:55:57,2020-08-15,81,Subscription (Annual) - Basic,True,73.240.11.122,"[210.0, 210.0]",32,420.00,brian.exile@gmail.com,32,810
17449,40,10,2019-11-04T16:40:44,2020-07-12,228,Subscription - Basic,True,24.231.191.6,"[21.25, 21.25, 21.25, 21.25, 21.25, 21.25, 21.25]",5,148.75,jl@lessnau.com,12,3481
17500,36,11,2019-11-17T08:04:56,2020-04-06,71,"Subscription - Basic, Feature: ASINs included ...",True,74.88.44.149,"[21.25, 8.5, 21.25, 8.5, 21.25, 8.5, 21.25, 8....",75,827.25,moshegarfinkel@gmail.com,74,3229
17827,32,8,2020-03-12T12:25:02,2020-09-11,151,"Feature: ASINs included in keyword - 10 ASINs,...",True,12.186.143.218,"[858.0, 10.0, 99.0]",127,967.00,TRADERMIKIAL@GMAIL.COM,175,51884
17286,32,8,2019-10-17T11:39:20,2019-10-21,99,"Subscription - Basic, Feature: ASINs included ...",True,24.70.101.173,"[25.0, 10.0, 0.0, 25.0, 10.0, 0.0, 25.0, 10.0,...",16,326.50,freshfieldnaturals@gmail.com,17,1452
17871,29,7,2020-04-12T17:23:47,2020-05-24,136,Subscription - Basic,True,73.138.65.214,"[25.0, 25.0, 25.0, 25.0, 10.0, 10.0, 18.75]",3,138.75,investxol@outlook.com,6,1910
17856,29,7,NaN,2020-06-05,131,Subscription - Starter,NaN,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0]",24,15.00,NaN,58,6265


In [27]:
actions_per_user_df[[
    'wc_email', 'wc_is_paying',
    'last_day_seen', 'num_keywords', 'num_asins',
    'num_days_active', 'num_weeks_active',
    'num_months_active', 'num_requests',
    'registered_at',
    'orders_total',
    'orders',
    'payments',
    'ip_address']].to_csv(os.path.join(OUT_DIR, 'all_actions.csv'))

In [24]:
# TODO
# 1. Correlate users by asins
# 2. Compure MRR and churn
# 3. Add time since registration
# 4. Visualize registrations

# Same IP

In [25]:
actions_per_user_df.groupby('ip_address').agg({'wc_email': 'nunique' }).sort_values(by='wc_email', ascending=False)

,wc_email
ip_address,
184.65.133.19,4
,1
46.211.53.252,1
69.115.36.12,1
69.123.220.152,1
69.124.11.115,1
70.169.227.87,1
70.182.6.26,1
70.190.113.252,1


In [26]:
actions_per_user_df[actions_per_user_df.ip_address=='184.65.133.19']

,num_weeks_active,num_months_active,registered_at,last_day_seen,num_days_active,orders,wc_is_paying,ip_address,payments,num_asins,orders_total,wc_email,num_keywords,num_requests
user_id,,,,,,,,,,,,,,
17517,8,5,2019-11-20T20:25:03,2019-11-25,22,Subscription (Trial only) - Starter,True,184.65.133.19,"[15.0, 0.0]",8,15.0,inquiry@hyponix.net,6,312
17616,2,1,2019-12-15T21:27:37,2019-12-20,4,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",2,25.0,mohijoj796@tmailer.org,4,49
17731,2,1,2020-01-23T18:49:08,2020-01-24,3,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",3,25.0,mavodix550@it-smart.org,16,99
17609,1,1,2019-12-13T23:36:49,2019-12-14,2,Subscription (Trial only) - Basic,True,184.65.133.19,"[25.0, 0.0]",1,25.0,pebiv71186@tmailer.org,5,31
